In [1]:
!pip install rouge-score evaluate -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux

In [2]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

# Thêm thêm Secret HF_TOKEN trong "Add-ons" => "Secrets" => Add Secret với label là "HF_TOKEN" và value là token của bạn (để chế độ WRITE trong HuggingFace)
secret_label = "HF_TOKEN"
secret_value = UserSecretsClient().get_secret(secret_label)
login(token=secret_value)

In [3]:
%%writefile train.py
import os
import argparse
import pandas as pd
from datasets import Dataset
import importlib
import transformers
importlib.reload(transformers)
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, Seq2SeqTrainingArguments
)
from datasets.utils.logging import disable_progress_bar, enable_progress_bar
from rouge_score import rouge_scorer
import torch

parser = argparse.ArgumentParser()
parser.add_argument("--task", type=str, choices=["gen", "denoise"], required=True)
parser.add_argument("--train_file", type=str, required=True)
parser.add_argument("--valid_file", type=str, required=True)
parser.add_argument("--output_dir", type=str, required=True)
parser.add_argument("--init_model", type=str, default="Salesforce/codet5-base")
parser.add_argument("--num_train_epochs", type=int, default=None)
parser.add_argument("--push_to_hub", action="store_true")
parser.add_argument("--hub_model_id", type=str, default=None)
args = parser.parse_args()

print(f"Loading model from: {args.init_model}")
tokenizer = AutoTokenizer.from_pretrained(args.init_model)
model = AutoModelForSeq2SeqLM.from_pretrained(args.init_model)

if args.task == "denoise":
    default_epochs = 3
    target_max_len = 64
elif args.task == "gen":
    default_epochs = 4
    target_max_len = 64

num_epochs = args.num_train_epochs if args.num_train_epochs else default_epochs

print(f"Loading data from: {args.train_file}")
df_train = pd.read_csv(args.train_file)
dataset_train = Dataset.from_pandas(df_train)

print(f"Loading data from: {args.valid_file}")
df_valid = pd.read_csv(args.valid_file)
dataset_valid = Dataset.from_pandas(df_valid)

def tokenize(example):
    model_input = tokenizer(
        example["content"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    labels = tokenizer(
        example["title"],
        truncation=True,
        padding="max_length",
        max_length=target_max_len
    )
    model_input["labels"] = labels["input_ids"]
    return model_input

print("Tokenizing...")
disable_progress_bar()
tokenized_dataset_train = dataset_train.map(tokenize)
tokenized_dataset_valid = dataset_valid.map(tokenize)
enable_progress_bar()

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(pred.strip().split(". ")) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split(". ")) for label in decoded_labels]

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = [scorer.score(pred, ref) for pred, ref in zip(decoded_preds, decoded_labels)]

    rouge1_f1 = sum(s["rouge1"].fmeasure for s in scores) / len(scores)
    rouge2_f1 = sum(s["rouge2"].fmeasure for s in scores) / len(scores)
    rougeL_f1 = sum(s["rougeL"].fmeasure for s in scores) / len(scores)

    return {
        "rouge1": rouge1_f1,
        "rouge2": rouge2_f1,
        "rougeL": rougeL_f1,
    }

training_args = Seq2SeqTrainingArguments(
    output_dir=args.output_dir,
    overwrite_output_dir=True,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=14,
    per_device_eval_batch_size=4,
    eval_accumulation_steps=4,
    generation_max_length=64,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir=f"{args.output_dir}/logs",
    logging_steps=100,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    learning_rate=3e-5,
    report_to="none",
    push_to_hub=args.push_to_hub,
    hub_model_id=args.hub_model_id,
    hub_strategy="end"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print(f"Starting training for task: {args.task} | Epochs: {num_epochs}")
import gc
gc.collect()
torch.cuda.empty_cache()
trainer.train()

if args.push_to_hub:
    trainer.push_to_hub()
    print(f"Model pushed to: https://huggingface.co/{args.hub_model_id}")

Writing train.py


In [4]:
# Bỏ 2 dòng cuối nếu không push lên HuggingFace
!python train.py \
  --task denoise \
  --train_file /kaggle/input/denoise-filler-v3/train_denoise.csv \
  --valid_file /kaggle/input/denoise-filler-v3/valid_denoise.csv \
  --output_dir /kaggle/working/denoise/ \
  --init_model Salesforce/codet5-base \
  --num_train_epochs 2 \
  --push_to_hub \
  --hub_model_id HuyTran1301/codeT5-phase1-ep2-v3

2025-06-05 13:44:18.878148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749131059.061628      52 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749131059.115279      52 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading model from: Salesforce/codet5-base
tokenizer_config.json: 100%|███████████████| 1.48k/1.48k [00:00<00:00, 11.1MB/s]
vocab.json: 100%|████████████████████████████| 703k/703k [00:00<00:00, 1.70MB/s]
merges.txt: 100%|████████████████████████████| 294k/294k [00:00<00:00, 33.4MB/s]
added_tokens.json: 100%|█████████████████████| 2.00/2.00 [00:00<00:00, 12.8kB/s]
special_tokens_map.json: 100%|█████████████| 12.5k/12.5k [00:00<00:00,